**The Activity Table shows the app installed and app puchase activity for spotify app along with country details**

In [0]:
CREATE table activity
(
user_id varchar(20),
event_name varchar(20),
event_date date,
country varchar(20)
);
delete from activity;
insert into activity values (1,'app-installed','2022-01-01','India')
,(1,'app-purchase','2022-01-02','India')
,(2,'app-installed','2022-01-01','USA')
,(3,'app-installed','2022-01-01','USA')
,(3,'app-purchase','2022-01-03','USA')
,(4,'app-installed','2022-01-03','India')
,(4,'app-purchase','2022-01-03','India')
,(5,'app-installed','2022-01-03','SL')
,(5,'app-purchase','2022-01-03','SL')
,(6,'app-installed','2022-01-04','Pakistan')
,(6,'app-purchase','2022-01-04','Pakistan');

num_affected_rows,num_inserted_rows
11,11


In [0]:
select * from activity;

user_id,event_name,event_date,country
1,app-installed,2022-01-01,India
1,app-purchase,2022-01-02,India
2,app-installed,2022-01-01,USA
3,app-installed,2022-01-01,USA
3,app-purchase,2022-01-03,USA
4,app-installed,2022-01-03,India
4,app-purchase,2022-01-03,India
5,app-installed,2022-01-03,SL
5,app-purchase,2022-01-03,SL
6,app-installed,2022-01-04,Pakistan


1. **Find total active users each day**

In [0]:
select
event_date,
count(distinct user_id) as users
from activity
group by event_date;

event_date,users
2022-01-01,3
2022-01-02,1
2022-01-03,3
2022-01-04,1


2. **Find total active users each week**

In [0]:
select
*,
date_part('week', event_date) as week_num
from 
activity;

user_id,event_name,event_date,country,week_num
1,app-installed,2022-01-01,India,52
1,app-purchase,2022-01-02,India,52
2,app-installed,2022-01-01,USA,52
3,app-installed,2022-01-01,USA,52
3,app-purchase,2022-01-03,USA,1
4,app-installed,2022-01-03,India,1
4,app-purchase,2022-01-03,India,1
5,app-installed,2022-01-03,SL,1
5,app-purchase,2022-01-03,SL,1
6,app-installed,2022-01-04,Pakistan,1


In [0]:
select
date_part('week', event_date) as week_num,
count(distinct user_id) as users
from 
activity
group by date_part('week', event_date) ;

week_num,users
52,3
1,4


3. **Date wise total number of users who made the purchase same day including all days**

In [0]:

select
user_id,
event_date
from 
activity
group by user_id,event_date having count(distinct event_name)=2


user_id,event_date
4,2022-01-03
5,2022-01-03
6,2022-01-04


In [0]:
select event_date,
count(distinct user_id)
from(select
user_id,
event_date
from 
activity
group by user_id,event_date having count(distinct event_name)=2) t
group by event_date

event_date,count(DISTINCTuser_id)
2022-01-03,2
2022-01-04,1



For all the dates


In [0]:
--select * from activity

select
event_date,
count(new_col) as users
from(
select user_id,
event_date,
count(distinct user_id) as users,
case when count(distinct event_name)=2 then user_id else null end as new_col
from
activity
group by user_id,event_date) group by event_date


event_date,users
2022-01-01,0
2022-01-02,0
2022-01-03,2
2022-01-04,1


4. **Percentage of Paid users in India, USA and any other countries should be tagged as Others**

Country Percentage_users

India   40

USA     40

Others  20

In [0]:
select * from activity

user_id,event_name,event_date,country
1,app-installed,2022-01-01,India
1,app-purchase,2022-01-02,India
2,app-installed,2022-01-01,USA
3,app-installed,2022-01-01,USA
3,app-purchase,2022-01-03,USA
4,app-installed,2022-01-03,India
4,app-purchase,2022-01-03,India
5,app-installed,2022-01-03,SL
5,app-purchase,2022-01-03,SL
6,app-installed,2022-01-04,Pakistan


In [0]:
select
*
from
activity
where event_name="app-purchase"

user_id,event_name,event_date,country
1,app-purchase,2022-01-02,India
3,app-purchase,2022-01-03,USA
4,app-purchase,2022-01-03,India
5,app-purchase,2022-01-03,SL
6,app-purchase,2022-01-04,Pakistan


In [0]:
select
*,
case when country IN ("India", "USA") then country else "Others" end as country_val
from activity

user_id,event_name,event_date,country,country_val
1,app-installed,2022-01-01,India,India
1,app-purchase,2022-01-02,India,India
2,app-installed,2022-01-01,USA,USA
3,app-installed,2022-01-01,USA,USA
3,app-purchase,2022-01-03,USA,USA
4,app-installed,2022-01-03,India,India
4,app-purchase,2022-01-03,India,India
5,app-installed,2022-01-03,SL,Others
5,app-purchase,2022-01-03,SL,Others
6,app-installed,2022-01-04,Pakistan,Others


In [0]:
with coun_val(select
*,
case when country IN ("India", "USA") then country else "Others" end as country_val
from activity where event_name="app-purchase"),

county_wise_users as(
select country_val,count(user_id) as users
from coun_val
group by country_val),

total as(
select
sum(users) as total_users
from county_wise_users)

select 
country_val as country,
(users/total_users)*100 as percentage_paid_users
from 
county_wise_users,total

country,percentage_paid_users
USA,20.0
India,40.0
Others,40.0


5. **Among all the users who installed the app on a given day, how many did in app puchase on the very next day ,  day wise result**


event_date           count_users

2022-01-01           0

2022-01-02           1 

2022-01-03           0

2022-01-04           0

In [0]:
select * from activity

user_id,event_name,event_date,country
1,app-installed,2022-01-01,India
1,app-purchase,2022-01-02,India
2,app-installed,2022-01-01,USA
3,app-installed,2022-01-01,USA
3,app-purchase,2022-01-03,USA
4,app-installed,2022-01-03,India
4,app-purchase,2022-01-03,India
5,app-installed,2022-01-03,SL
5,app-purchase,2022-01-03,SL
6,app-installed,2022-01-04,Pakistan


In [0]:
with prev_data as (select
*, 
lag(event_name,1) over(partition by user_id order by event_date) as prev_event_name,
lag(event_date,1) over(partition by user_id order by event_date) as prev_event_date
from activity)

select * from prev_data where event_name="app-purchase" and prev_event_name="app-installed" and datediff(event_date,prev_event_date)=1


user_id,event_name,event_date,country,prev_event_name,prev_event_date
1,app-purchase,2022-01-02,India,app-installed,2022-01-01


In [0]:
with prev_data as (select
*, 
lag(event_name,1) over(partition by user_id order by event_date) as prev_event_name,
lag(event_date,1) over(partition by user_id order by event_date) as prev_event_date
from activity),
final_data as (
select event_date,count(distinct user_id) as no_of_users from prev_data 
where event_name="app-purchase" and prev_event_name="app-installed" and datediff(event_date,prev_event_date)=1
group by event_date)

select activity.event_date,nvl(sum(no_of_users),0) as total_users from activity left join final_data on activity.event_date=final_data.event_date
group by activity.event_date
order by activity.event_date 

event_date,total_users
2022-01-01,0
2022-01-02,1
2022-01-03,0
2022-01-04,0
